In [1]:
import requests 
import open3d as o3d
import os

In [5]:
os.environ['http_proxy'] = "http://proxy.robotron.de:80" 
os.environ['https_proxy'] = "https://proxy.robotron.de:80" 

In [6]:
url = "https://rcv.westeurope.cloudapp.azure.com/3dodserving/api/models"

payload = {}
files = {}
headers = {
  'Content-Type': 'multipart/form-data; boundary=--------------------------555221334256103757881987'
}

response = requests.request("GET", url, headers=headers, data = payload, files = files)

print(response.text.encode('utf8'))

b'[{"id":"1","name":"box3d","type":"3d_detection"}]'


In [6]:
url = "http://localhost:8080/api/detect?model_id=1"

files = {'file': open('0005.ply','rb')}

response = requests.post(url,  files = files)

response.json()

[{'class': 'box',
  'confidence': 0.9892852902412415,
  'coordinates': {'0': [0.3100882275297897,
    1.9325232134099077,
    -0.08131351548609847],
   '1': [0.5125316754284144, 1.3815603723767127, -0.06913612611849415],
   '2': [-0.17065788342502283, 1.1306546676927653, -0.06359060429208575],
   '3': [-0.3731013313236475, 1.6816175087259604, -0.07576799365969006],
   '4': [0.3100882275297897, 1.944366604004498, 0.4545376372100998],
   '5': [0.5125316754284144, 1.393403762971303, 0.46671502657770414],
   '6': [-0.17065788342502283, 1.1424980582873556, 0.4722605484041125],
   '7': [-0.3731013313236475, 1.6934608993205507, 0.4600831590365082]}}]

In [7]:
print('class: ' + str(response.json()[0]['class_id']))
print('confidence: ' + str(response.json()[0]['probability']))

KeyError: 'class_id'

In [8]:
points = []
for i in range(0,8):
    points.append(response.json()[0]['coordinates'][str(i)])
    i +=1

lines = [
        [0, 1],
        [0, 3],
        [1, 5],
        [3, 7],
        [4, 0],
        [4, 5],
        [5, 6],
        [6, 7],
        [2, 3],
        [2, 6],
        [2, 1],
        [4, 7]
        ]

colors = [[1, 0, 0] for i in range(len(lines))]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(points),
    lines=o3d.utility.Vector2iVector(lines),
)
line_set.colors = o3d.utility.Vector3dVector(colors)

pcd = o3d.io.read_point_cloud('0005_cl.ply')
o3d.visualization.draw_geometries([line_set, pcd])    